In [2]:
import pandas as pd

# Cargar el dataset
df = pd.read_csv("url_spam.csv")

# Ver primeras filas
df.head()


,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True


In [3]:
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

# Preprocesamiento de URLs
def preprocess_url(url):
    # Minúsculas
    url = url.lower()
    # Tokenización simple por signos de puntuación y slashes
    tokens = re.split(r'\W+', url)
    # Eliminar stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and word.isalpha()]
    # Lematización
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

# Aplicar preprocesamiento
df['clean_url'] = df['url'].apply(preprocess_url)

# División de datos
X = df['clean_url']
y = df['is_spam'].astype(int)  # convertir booleano a int

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/beacastro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/beacastro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

# Pipeline: TF-IDF + SVM
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC())
])

# Entrenamiento
pipeline.fit(X_train, y_train)

# Evaluación
y_pred = pipeline.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[435  20]
 [ 20 125]]
              precision    recall  f1-score   support

           0       0.96      0.96      0.96       455
           1       0.86      0.86      0.86       145

    accuracy                           0.93       600
   macro avg       0.91      0.91      0.91       600
weighted avg       0.93      0.93      0.93       600



In [5]:
from sklearn.model_selection import GridSearchCV

# Grid de hiperparámetros
param_grid = {
    'svm__C': [0.1, 1, 10],
    'svm__kernel': ['linear', 'rbf'],
    'svm__gamma': ['scale', 'auto']
}

# GridSearch con 5-fold CV
grid = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)
grid.fit(X_train, y_train)

# Resultados
print("Mejor combinación:", grid.best_params_)
print("Mejor score:", grid.best_score_)

# Evaluación final con el mejor modelo
y_pred_optimized = grid.predict(X_test)
print(confusion_matrix(y_test, y_pred_optimized))
print(classification_report(y_test, y_pred_optimized))


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...svm__C=0.1, svm__gamma=scale, svm__kernel=linear; total time=   0.2s
[CV] END ...svm__C=0.1, svm__gamma=scale, svm__kernel=linear; total time=   0.2s
[CV] END ....svm__C=0.1, svm__gamma=auto, svm__kernel=linear; total time=   0.2s
[CV] END ...svm__C=0.1, svm__gamma=scale, svm__kernel=linear; total time=   0.2s
[CV] END ...svm__C=0.1, svm__gamma=scale, svm__kernel=linear; total time=   0.2s
[CV] END .....svm__C=1, svm__gamma=scale, svm__kernel=linear; total time=   0.2s
[CV] END ...svm__C=0.1, svm__gamma=scale, svm__kernel=linear; total time=   0.2s
[CV] END .....svm__C=1, svm__gamma=scale, svm__kernel=linear; total time=   0.2s
[CV] END ....svm__C=0.1, svm__gamma=auto, svm__kernel=linear; total time=   0.2s
[CV] END .......svm__C=0.1, svm__gamma=auto, svm__kernel=rbf; total time=   0.2s
[CV] END .......svm__C=0.1, svm__gamma=auto, svm__kernel=rbf; total time=   0.2s
[CV] END ......svm__C=0.1, svm__gamma=scale, svm

In [6]:
import joblib

# Guardar modelo optimizado
joblib.dump(grid.best_estimator_, 'modelo_url_spam.pkl')


['modelo_url_spam.pkl']

In [7]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Predicciones sobre el set de test
y_pred_test = grid.predict(X_test)

# Métricas de evaluación
acc = accuracy_score(y_test, y_pred_test)
cm = confusion_matrix(y_test, y_pred_test)
report = classification_report(y_test, y_pred_test)

print(f"Accuracy: {acc:.4f}")
print("\nMatriz de Confusión:")
print(cm)
print("\nReporte de Clasificación:")
print(report)


Accuracy: 0.9467

Matriz de Confusión:
[[436  19]
 [ 13 132]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.97      0.96      0.96       455
           1       0.87      0.91      0.89       145

    accuracy                           0.95       600
   macro avg       0.92      0.93      0.93       600
weighted avg       0.95      0.95      0.95       600



Evaluación del Modelo – Interpretación Técnica
Matriz de Confusión

    [[436   19]
    [ 13    132]]

    436 ejemplos de clase 0 (no spam) fueron correctamente clasificados.
    132 ejemplos de clase 1 (spam) fueron correctamente clasificados.
    19 ejemplos de clase 0 fueron clasificados incorrectamente como spam (falsos positivos).
    13 ejemplos de clase 1 fueron clasificados incorrectamente como no spam (falsos negativos).

Métricas por Clase
Clase	            Precision	Recall	F1-Score	Support
0 (no spam)	            0.97	0.96	0.96	      455
1 (spam)	            0.87	0.91	0.89	      145

    La clase 0 tiene métricas más altas, lo que puede deberse a su mayor representación en los datos (mayor soporte).
    La clase 1, aunque minoritaria, presenta una precisión aceptable (0.87) y un recall elevado (0.91), lo cual es positivo para evitar dejar pasar spam.

Promedios Globales
    Accuracy: 0.9467 → El modelo acertó aproximadamente en el 94.67% de los casos.
    Macro promedio: 0.93 de F1-score → Promedio simple entre clases, indicando un rendimiento balanceado.
    Promedio ponderado (weighted avg): 0.95 de F1-score → Ajustado según el número de ejemplos por clase, confirma el buen desempeño general.

Conclusión

El modelo SVM optimizado logra una buena generalización sobre el conjunto de test, con alta precisión y recall especialmente en la clase de interés (spam). El número de falsos negativos es bajo (13), lo cual es favorable en contextos donde se busca minimizar el riesgo de que spam pase desapercibido.